In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from pytz import timezone
from dateutil import parser
import calendar

In [13]:
day = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

def clean_data_comb_credit():
  df_chase = pd.read_csv("chase.csv")
  df_chase.drop(['Post Date', 'Memo'], axis=1, inplace=True)
  df_chase.rename(columns={'Transaction Date': 'Date', 'Description': 'Note', 'Category': 'Category name'}, inplace=True)

  df_apple = pd.read_csv("apple.csv")
  df_apple.drop(['Clearing Date', 'Description', 'Purchased By'], axis=1, inplace=True)
  df_apple.rename(columns={'Transaction Date': 'Date', 'Merchant': 'Note', 'Category': 'Category name', 'Amount (USD)': 'Amount'}, inplace=True)
  df_apple['Amount'] = df_apple['Amount'].apply(lambda amt: -1*amt)

  df_discover = pd.read_csv("discover.csv")
  df_discover.drop(['Post Date'], axis=1, inplace=True)
  df_discover.rename(columns={'Trans. Date': 'Date', 'Description': 'Note', 'Category': 'Category name'}, inplace=True)
  df_discover['Amount'] = df_discover['Amount'].apply(lambda amt: -1*amt)
  df_discover = df_discover.loc[df_discover['Note'] != 'INTERNET PAYMENT - THANK YOU']

  return pd.concat([df_chase, df_apple, df_discover])


def splitDate(date):
    if '-' in date:
      yyyy = date[:4]
      mm = date[5:7]
      dd = date[8:]
    else:
      mm = date[:2]
      dd = date[3:5]
      yyyy = date[6:]
    return yyyy, mm, dd

def convert(date):
  if '-' in date:
    return date
  yyyy, mm, dd = splitDate(date)
  return yyyy + '-' + mm + '-' + dd

def getDay(date):
  yyyy, mm, dd = splitDate(date)
  return day[datetime(int(yyyy), int(mm), int(dd)).weekday()]


def add_note_splitwise(row, who):
  note = ''
  if row['Amount'] < 0:
    note = f" ({row['Cost']} paid by {who})"
  else:
    note = f" ({row['Cost']} paid by You)"
  return row['Note'] + note


def get_splitwise_data(month, filename):
  if filename == 'jaya.csv':
    who = 'Jaya'
  else:
    who = 'Isha'

  hh = pd.read_csv(filename)
  hh = hh.reset_index()
  hh.columns = hh.iloc[0]
  hh = hh.iloc[1:]
  # '2024-01'
  hh = hh[hh['Date'].map(lambda x: x[:7]) == month]
  hh = hh[['Date', 'Description', 'Category', 'Cost', 'Simran Bhake']]
  hh.rename(columns={'Description': 'Note', 'Category': 'Category name', 'Simran Bhake': 'Amount'}, inplace=True)
  hh['Amount'] = hh['Amount'].apply(lambda amt: float(amt)) # converting to number
  hh['Note'] = hh.apply(lambda row: add_note_splitwise(row, who), axis=1)
  hh.drop('Cost', axis=1, inplace=True)
  hh = hh[hh["Category name"] != 'Payment']
  hh = hh[hh["Amount"] != 0]
  hh['Amount'] = hh['Amount'].apply(lambda amt: -abs(amt)) # convert all vals to -ve
  hh["Day"] = hh['Date'].apply(lambda date: getDay(date))
  return hh


def isVacayDay(date, vacay):
  start = int(vacay[0])
  end = int(vacay[1])
  if(start <= int(date[8:]) and int(date[8:]) <= end):
    return True
  return False


def isWeekday(note, row_day):
  if len(note) != 0:
    for d in day:
      if d.lower() in note:
        return True

  if len(row_day) != 0:
    if row_day in day[:5]:
      return True

  return False


def fixCategory(row, vacay):
  cat = row['Category name']
  row_note = row['Note'].lower()
  is_weekday = row['Day'] in day[:5]
  # if(cat == 'Travel' or cat == 'Transportation' or cat == 'Taxi'):
  if 'uber' in row_note or 'lyft' in row_note or 'cab' in row_note:
    if(isWeekday(row_note, row['Day']) and not isVacayDay(row['Date'], vacay)):
      return 'Cab for Work'
    else:
      return 'Cab'
  elif 'MTA' in row['Note']:
    return 'Transport'

  if ('amazon prime' in row_note) or ('prime video' in row_note) or (cat == 'Services' and 'tmobile' in row_note) or (cat == 'Electricity'):
    return 'Bills & Fees'

  if(cat == 'Transportation'):
    return 'Transport'

  if(cat == 'Health & Wellness' or cat == 'Merchandise'):
    return 'Shopping'

  if(cat == 'Restaurants' or cat == 'Food & Drink'):
    if(isWeekday("", row['Day']) and not isVacayDay(row['Date'], vacay)):
      return 'Food at Work'
    else:
      return 'Food & Drink'

  # if(cat == 'Services' and 'tmobile' in row_note):
  #   # row['Amount'] = -20.39
  #   return 'Bills & Fees'

  if(cat in ["General", "Grocery", "Supermarkets"]):
    return 'Groceries'

  if(cat == 'Household supplies' or cat == 'Cleaning'):
    return 'Home'

  return cat



def get_csv_for_spendee_upload(month, splitwiseFiles, vacay):
  df = clean_data_comb_credit()

  df["Day"] = df['Date'].apply(lambda date: getDay(date))
  df['Date'] = df['Date'].apply(lambda date: convert(date))

  # '2024-01'
  df = df[df['Date'].map(lambda x: x[:7]) == month]

  for filename in splitwiseFiles:
    hh = get_splitwise_data(month, filename)
    df = pd.concat([df, hh])

  df = df[['Day', 'Date', 'Note', 'Category name', 'Type', 'Amount']]

  df = df.loc[df['Type'] != 'Payment']
  df.loc[df.Amount < 0, 'Type'] = "Expense"
  df.loc[df.Amount > 0, ['Type', 'Category name']] = "Income", "Other"

  df.sort_values(by='Date', inplace=True)
  df.reset_index(drop=True, inplace=True)

  df["Category name"] = df.apply(lambda row: fixCategory(row, vacay), axis=1)

  work_subway = df.loc[(df['Category name'] == "Transport") & (df['Day'].isin(day[:5])), 'Amount'].sum()
  df.drop(df[(df["Category name"] == "Transport") & (df['Day'].isin(day[:5]))].index, inplace=True)

  df.reset_index(drop=True, inplace=True)

  df.loc[len(df)] = [df["Day"][len(df)-1], df["Date"][len(df)-1], 'For work', 'Transport', 'Expense', round(work_subway, 2)]

  df['Note'] = df['Note'].apply(lambda x: x.title())

  return df


In [ ]:
# get_splitwise_data('2024-03', 'jaya.csv')

In [14]:
month = '2024-05'

# ['household.csv', 'jaya.csv'], vacay []
df = get_csv_for_spendee_upload(month, [], ['14', '17'])

df.head(300)

,Day,Date,Note,Category name,Type,Amount
0,Wed,2024-05-01,Chipotle 2375,Food at Work,Expense,-12.74
1,Thu,2024-05-02,Lyft *Ride Wed 5Pm,Cab for Work,Expense,-21.99
2,Thu,2024-05-02,American Air0012137767914,Travel,Expense,-206.20
3,Fri,2024-05-03,Omar'S Mediterranean,Food at Work,Expense,-16.34
4,Sat,2024-05-04,Ic* Instacart,Groceries,Expense,-41.43
...,...,...,...,...,...,...
73,Wed,2024-05-29,Lyft *Citi Bike Ride,Cab for Work,Expense,-9.79
74,Thu,2024-05-30,Omar'S Mediterranean,Food at Work,Expense,-16.34
75,Fri,2024-05-31,Monthly Installments (6 Of 6),Installment,Expense,-29.85
76,Fri,2024-05-31,Lyft *Ride Thu 5Pm,Cab for Work,Expense,-24.70


In [15]:
df.drop(['Day'], axis=1, inplace=True)

outputFileName = calendar.month_abbr[int(month[5:])] + '_' + month[:4] + ".csv"

df.to_csv(outputFileName, index=False)